In [25]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [26]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # max context length for predictions.
max_iters = 6000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"{device=}")
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

device='cuda'


In [27]:
torch.manual_seed(1337)

# import dataset
with open('../data/goethe.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# # unique characters that occur in the dataset
# chars = sorted(list(set(text)))
# vocab_size = len(chars)
# print(f"{vocab_size=}")

# # create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")

# Replace `encode` and `decode`:
encode = lambda s: tokenizer.encode(s, add_special_tokens=False)
decode = lambda l: tokenizer.decode(l)

vocab_size = tokenizer.vocab_size


# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [28]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPT(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [29]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [30]:
model = GPT()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# Adam optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


49.392729 M parameters


In [31]:
from torch.utils.tensorboard import SummaryWriter

# Create TensorBoard writer 
writer = SummaryWriter(log_dir="../runs/goethe_stats")

# ------- Training loop
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_loss = losses['train']
        val_loss = losses['val']
        print(f"step {iter}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")

        # Log to TensorBoard
        writer.add_scalar("loss/train", train_loss, iter)
        writer.add_scalar("loss/val", val_loss, iter)

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# save the model
model_name = "GöPTv3.0"
torch.save(model.state_dict(), f"../models/{model_name}.pt")

# Close the writer
writer.close()

step 0: train loss 10.9881, val loss 10.9886
step 500: train loss 5.5143, val loss 5.6873
step 1000: train loss 4.9188, val loss 5.4105
step 1500: train loss 4.4884, val loss 5.2830
step 2000: train loss 4.1537, val loss 5.2567
step 2500: train loss 3.8691, val loss 5.2987
step 3000: train loss 3.5995, val loss 5.3714
step 3500: train loss 3.3525, val loss 5.4667
step 4000: train loss 3.1292, val loss 5.5809
step 4500: train loss 2.9177, val loss 5.6753
step 5000: train loss 2.7262, val loss 5.8233
step 5500: train loss 2.5535, val loss 5.9428
step 5999: train loss 2.3914, val loss 6.0721


In [32]:
# load saved model
model = GPT()
model.load_state_dict(torch.load(f"../models/{model_name}.pt"))
m = model.to(device)

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

<s>Blagen, Zu dringen Im Heilandlebung.
Halt sogleich jener Mühmüstrung, Arkadle Beneiden, Verklärenden Rat, dicht welche des Orkus frommen Schöne Getrenntes, eigenstamme Kühlt es sicher dich alsobald niedergesehen?
Doch was du umgibt?
Man macht wohl, wie verworren, Vergänglich verschüchtert besser erscheint Mit frommer Schätzen holdung; Doch Obdach ist man besser viel von Ort; Doch oft sieht man, wie man bös lernen.
Ein Gleichnis aber doch bald verstehen, als er nach Spekulation vermehrt Schwester zieht, An Vater kaum; Nie erscheint er würdig, die alten Gestalten Mit Lotten umständlich sind.
Was sie sind auch bei Hof zu ficht hinüber?
Hat er sich selbst In aller Weisheit wächst dies alles Schonung gedeckt; Es fällt ein großes Glück und ähnlich, Liebenderbt, Wie noch Wunder uns je gedrungen, So selten kann der Anblick bereiten, Nach Tat ein Weileins dem Boden widmen, Ein eigner, Fort Feuer; Selbst dann bedarf keiner Am alten Haupt auch als des Königreichs stols eignen ersten Eintritt.
